# Main Program for Cross Validation

Author: Efraim Manurung

MSc Thesis in Information Technology Group, Wageningen University

efraim.manurung@gmail.com

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [15]:
# Load the datasets from separate files
file_path = r"iot-datasets-train-dnn-parameters.xlsx"

# Load and clean the training dataset
mgh_data = pd.read_excel(file_path)

# Display the first few rows of the dataframe
mgh_data.head()

,time,global out,global in,temp in,temp out,rh in,rh out,co2 in,co2 out,toplights,ventilation,heater,Unnamed: 12,Unnamed: 13
0,0,0.032943,0.0,19.806459,22.685000,62.243682,52.775000,409.000000,1530.5,1,0.0,0,dataset3,May
1,300,0.032943,0.0,19.806459,22.685000,62.243682,52.775000,409.000000,1530.5,1,0.0,0,NaN,NaN
2,600,0.032943,0.0,19.889053,22.745000,62.040377,52.510000,409.000000,1506.5,1,0.0,0,NaN,NaN
3,900,0.032943,0.0,20.017409,22.823333,61.694059,52.213333,408.333333,1455.0,1,0.0,0,NaN,NaN
4,1200,0.032943,0.0,20.145762,22.865000,61.322003,52.015000,408.000000,1413.0,1,0.0,0,NaN,NaN


In [16]:
def r2_score_metric(y_true, y_pred):
    """Custom R2 score metric"""
    SS_res =  tf.reduce_sum(tf.square(y_true - y_pred)) 
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true))) 
    return (1 - SS_res/(SS_tot + tf.keras.backend.epsilon()))

def build_model(input_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))  # More neurons
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))  # Additional hidden layer
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))  # Additional hidden layer
    model.add(Dropout(0.2))
    model.add(Dense(1))
    
    adam = Adam(learning_rate=0.001, use_ema=True, ema_momentum=0.9)
    model.compile(optimizer=adam, loss='mean_squared_error', metrics=['mae', r2_score_metric])
    
    return model

def cross_validate_model(target_variable, _epochs=100, _batch_size=10):
    features = ['time', 'global out', 'temp out', 'rh out', 'co2 out', 'ventilation', 'toplights', 'heater']
    
    X = mgh_data[features]
    y = mgh_data[target_variable]

    # Initialize KFold with 10 splits
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    rmse_scores = []
    mae_scores = []
    r2_scores = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        # Initialize and fit the scaler on the training data
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        model = build_model(X_train_scaled.shape[1])
        
        model.fit(X_train_scaled, y_train, epochs=_epochs, batch_size=_batch_size, verbose=1)
        
        y_val_pred = model.predict(X_val_scaled)

        mse = mean_squared_error(y_val, y_val_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_val, y_val_pred)
        r2 = r2_score(y_val, y_val_pred)
        
        rmse_scores.append(rmse)
        mae_scores.append(mae)
        r2_scores.append(r2)
    
    print("Target variable : ", target_variable)
    print("10-Fold Cross-Validation Scores:")
    for i, (rmse, mae, r2) in enumerate(zip(rmse_scores, mae_scores, r2_scores)):
        print(f"Fold {i+1}: RMSE = {rmse:.4f}, MAE = {mae:.4f}, R2 = {r2:.4f}")
    
    print(f"\nAverage RMSE: {np.mean(rmse_scores):.4f}")
    print(f"Average MAE: {np.mean(mae_scores):.4f}")
    print(f"Average R2: {np.mean(r2_scores):.4f}")

In [17]:
# List of target variables
target_variables = ['global in', 'temp in', 'rh in', 'co2 in']

# Iterate through each target variable and call the function
for target in target_variables:
    cross_validate_model(target, _epochs = 1, _batch_size= 4)

c:\Users\frm19\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4492/4492 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 1.6380 - mae: 0.6639 - r2_score_metric: -2.2341
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


c:\Users\frm19\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4492/4492 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 1.5678 - mae: 0.6595 - r2_score_metric: -2.1418
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


c:\Users\frm19\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

4492/4492 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 23002.5117 - mae: 96.3631 - r2_score_metric: -354597797888.0000
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


c:\Users\frm19\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4492/4492 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 22459.4922 - mae: 97.3438 - r2_score_metric: -334416510976.0000
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


c:\Users\frm19\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4492/4492 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 21204.9629 - mae: 92.6652 - r2_score_metric: -309537046528.0000
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Target variable :  co2 in
10-Fold Cross-Validation Scores:
Fold 1: RMSE = 16.0665, MAE = 13.0839, R2 = -13.2184
Fold 2: RMSE = 19.0506, MAE = 16.3898, R2 = -32.5898
Fold 3: RMSE = 19.9459, MAE = 17.3650, R2 = -7.9663
Fold 4: RMSE = 11.7996, MAE = 8.9238, R2 = -4.0823
Fold 5: RMSE = 37.7491, MAE = 11.7911, R2 = -0.1685
Fold 6: RMSE = 20.4085, MAE = 9.2712, R2 = -0.3846
Fold 7: RMSE = 15.3925, MAE = 12.1158, R2 = -3.6638
Fold 8: RMSE = 14.1804, MAE = 11.7388, R2 = -8.6866
Fold 9: RMSE = 17.7360, MAE = 14.2948, R2 = -5.2192
Fold 10: RMSE = 16.4375, MAE = 13.2307, R2 = -15.2386

Average RMSE: 18.8767
Average MAE: 12.8205
Average R2: -9.1218


# BEST RESULTS
with _epochs = 10, and bath_size = 10

In [20]:
# List of target variables
target_variables = ['global in', 'temp in', 'rh in', 'co2 in']

# Iterate through each target variable and call the function
for target in target_variables:
    cross_validate_model(target, _epochs = 10, _batch_size= 5)

Epoch 1/10


c:\Users\frm19\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3593/3593 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 1.7422 - mae: 0.6967 - r2_score_metric: 0.0744
Epoch 2/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.7395 - mae: 0.4324 - r2_score_metric: 0.6349
Epoch 3/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.7450 - mae: 0.4253 - r2_score_metric: 0.6822
Epoch 4/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.6530 - mae: 0.3930 - r2_score_metric: 0.6518
Epoch 5/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.5587 - mae: 0.3769 - r2_score_metric: 0.7065
Epoch 6/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.5672 - mae: 0.3663 - r2_score_metric: 0.7334
Epoch 7/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 4s 986us/step - loss: 0.5747 - mae: 0.3690 - r2_score_metric: 0.7627
Epoch 8/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 4s 996us/step - loss: 0.4987 - mae: 0.3573 - r2_score_metric: 0.7510
Epoch 9/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.5514 - mae: 0.3650 - r2_score_metric: 0.7534
Epoch 10/10
3593/3593 ━━━━

c:\Users\frm19\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3593/3593 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - loss: 1.4874 - mae: 0.6578 - r2_score_metric: 0.0901
Epoch 2/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.8191 - mae: 0.4404 - r2_score_metric: 0.5960
Epoch 3/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 3s 960us/step - loss: 0.7192 - mae: 0.4099 - r2_score_metric: 0.6173
Epoch 4/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 4s 974us/step - loss: 0.6804 - mae: 0.3922 - r2_score_metric: 0.6720
Epoch 5/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 3s 940us/step - loss: 0.6224 - mae: 0.3824 - r2_score_metric: -3.0846
Epoch 6/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 4s 971us/step - loss: 0.5791 - mae: 0.3590 - r2_score_metric: 0.5873
Epoch 7/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 3s 950us/step - loss: 0.6004 - mae: 0.3668 - r2_score_metric: 0.7105
Epoch 8/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 3s 954us/step - loss: 0.5866 - mae: 0.3678 - r2_score_metric: 0.7460
Epoch 9/10
3593/3593 ━━━━━━━━━━━━━━━━━━━━ 4s 973us/step - loss: 0.6597 - mae: 0.3712 - r2_score_metric: 0.6219
Epoch 10/10
35

...

3594/3594 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 23067.9199 - mae: 97.9405 - r2_score_metric: -345874923520.0000
Epoch 2/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 3551.9553 - mae: 45.9787 - r2_score_metric: -49219690496.0000
Epoch 3/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 3151.6111 - mae: 44.0336 - r2_score_metric: -45706178560.0000
Epoch 4/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 2987.5066 - mae: 43.2430 - r2_score_metric: -44657266688.0000
Epoch 5/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 2883.5879 - mae: 42.4470 - r2_score_metric: -44084523008.0000
Epoch 6/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 2907.7837 - mae: 41.9308 - r2_score_metric: -39219179520.0000
Epoch 7/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 2927.0713 - mae: 41.2991 - r2_score_metric: -41302560768.0000
Epoch 8/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 2705.7913 - mae: 40.7397 - r2_score_metric: -35548942336.0000
Epoch 9/10
3594/3

c:\Users\frm19\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3594/3594 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 25615.1250 - mae: 104.5492 - r2_score_metric: -435887603712.0000
Epoch 2/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 4271.4790 - mae: 48.7904 - r2_score_metric: -53353766912.0000
Epoch 3/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 3602.8367 - mae: 47.6816 - r2_score_metric: -52439568384.0000
Epoch 4/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 3671.3940 - mae: 46.5942 - r2_score_metric: -51788435456.0000
Epoch 5/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 3473.5176 - mae: 46.4280 - r2_score_metric: -53626925056.0000
Epoch 6/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 3330.3611 - mae: 45.8215 - r2_score_metric: -49222119424.0000
Epoch 7/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 3389.8860 - mae: 46.2189 - r2_score_metric: -50301886464.0000
Epoch 8/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 3451.9202 - mae: 45.5956 - r2_score_metric: -50699620352.0000
Epoch 9/10
3594/

c:\Users\frm19\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3594/3594 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 24703.6543 - mae: 101.9257 - r2_score_metric: -406512893952.0000
Epoch 2/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 3477.5562 - mae: 46.7150 - r2_score_metric: -50807312384.0000
Epoch 3/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 3385.7053 - mae: 45.2119 - r2_score_metric: -48583593984.0000
Epoch 4/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 3431.3867 - mae: 45.5400 - r2_score_metric: -50046353408.0000
Epoch 5/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 3185.3101 - mae: 44.9868 - r2_score_metric: -48147460096.0000
Epoch 6/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 3216.0811 - mae: 44.1247 - r2_score_metric: -45800697856.0000
Epoch 7/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 3011.0366 - mae: 43.0187 - r2_score_metric: -47626821632.0000
Epoch 8/10
3594/3594 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 3049.4131 - mae: 43.4405 - r2_score_metric: -42952921088.0000
Epoch 9/10
3594/